<a href="https://colab.research.google.com/github/fjt067316/Kuzushiji_mnist/blob/main/Kmnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install torch torchvision

In [34]:
from torch.nn import Module
from torch.nn import Conv2d
from torch.nn import Linear
from torch.nn import MaxPool2d
from torch.nn import ReLU
from torch.nn import LogSoftmax
from torch import flatten

class kmnist_classifier(nn.Module):
  def __init__(self, numChannels, nclasses):
    super(kmnist_classifier, self).__init__()

    self.conv1 = Conv2d(numChannels, out_channels=20, kernel_size=(5,5)) # in_channels = depth of image rgb = 3 (= depth of kernel?), out channels = number of kernels ie one output per kernel
    self.relu1 = ReLU() # 24x24
    self.maxpool1 = MaxPool2d(kernel_size=(2,2), stride=2) #12x12

    self.conv2 = Conv2d(in_channels=20, out_channels=50, kernel_size=(5,5)) # in_channels = depth of image rgb = 3 (= depth of kernel?), out channels = number of kernels ie one output per kernel
    self.relu2 = ReLU() # 8x8
    self.maxpool2 = MaxPool2d(kernel_size=(2,2), stride=2) # 50x4x4

    self.fcl1 = Linear(in_features=800, out_features=500)
    self.relu3 = ReLU()

    self.fcl2 = Linear(in_features=500, out_features=nclasses)
    self.logSoftmax = LogSoftmax(dim=1)

  def forward(self, x):
    x = self.conv1(x)
    x = self.relu1(x)
    x = self.maxpool1(x)

    x = self.conv2(x)
    x = self.relu2(x)
    x = self.maxpool2(x)

    x = flatten(x, 1)

    x = self.fcl1(x)
    x = self.relu3(x)

    x = self.fcl2(x)
    output = self.logSoftmax(x)

    return output

In [35]:
import matplotlib
matplotlib.use("Agg")

from sklearn.metrics import classification_report
from torch.utils.data import random_split
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
from torchvision.datasets import KMNIST
from torch.optim import Adam
from torch import nn
import matplotlib.pyplot as plt
import numpy as np
import argparse
import torch
import time

# define training hyperparameters
INIT_LR = 1e-3
BATCH_SIZE = 64
EPOCHS = 10
# define the train and val splits
TRAIN_SPLIT = 0.75
VAL_SPLIT = 1 - TRAIN_SPLIT
# set the device we will be using to train the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("[INFO] loading the KMNIST dataset...")
trainData = KMNIST(root="data", train=True, download=True,
	transform=ToTensor())
testData = KMNIST(root="data", train=False, download=True,
	transform=ToTensor())

[INFO] loading the KMNIST dataset...


In [36]:
# calculate the train/validation split
print("[INFO] generating the train/validation split...")
numTrainSamples = int(len(trainData) * TRAIN_SPLIT)
numValSamples = int(len(trainData) * VAL_SPLIT)
(trainData, valData) = random_split(trainData,
	[numTrainSamples, numValSamples],
	generator=torch.Generator().manual_seed(42))

# initialize the train, validation, and test data loaders
trainDataLoader = DataLoader(trainData, shuffle=True,
	batch_size=BATCH_SIZE)
valDataLoader = DataLoader(valData, batch_size=BATCH_SIZE)
testDataLoader = DataLoader(testData, batch_size=BATCH_SIZE)
# calculate steps per epoch for training and validation set
trainSteps = len(trainDataLoader.dataset) // BATCH_SIZE
valSteps = len(valDataLoader.dataset) // BATCH_SIZE

[INFO] generating the train/validation split...


In [37]:
print("initializing model")

# model = kmnist_classifier(numChannels=1, nclasses=len(trainData.dataset.classes)).to(device)

model = kmnist_classifier(
	numChannels=1,
	nclasses=len(trainData.dataset.classes)).to(device)

# initialize our optimizer and loss function
opt = Adam(model.parameters(), lr=INIT_LR)
lossFn = nn.NLLLoss()

# initialize a dictionary to store training history
H = {
	"train_loss": [],
	"train_acc": [],
	"val_loss": [],
	"val_acc": []
}
# measure how long training is going to take
print("[INFO] training the network...")
startTime = time.time()

initializing model
[INFO] training the network...


In [49]:
# loop over our epochs
for e in range(0, EPOCHS):
	# set the model in training mode
	model.train()
	# initialize the total training and validation loss
	totalTrainLoss = 0
	totalValLoss = 0
	# initialize the number of correct predictions in the training
	# and validation step
	trainCorrect = 0
	valCorrect = 0
	# loop over the training set
	for (x, y) in trainDataLoader:
		# send the input to the device
		(x, y) = (x.to(device), y.to(device))
		# perform a forward pass and calculate the training loss
		pred = model(x)
		loss = lossFn(pred, y)
		# zero out the gradients, perform the backpropagation step,
		# and update the weights
		opt.zero_grad() # reset gradients for batch
		loss.backward()
		opt.step()
		# add the loss to the total training loss so far and
		# calculate the number of correct predictions
		totalTrainLoss += loss
		trainCorrect += (pred.argmax(1) == y).type(
			torch.float).sum().item()
	print(trainCorrect / len(trainDataLoader.dataset))


0.9992888888888889
0.9987111111111111
0.9978
0.9979111111111111
0.9994444444444445
0.9991111111111111
0.9984444444444445
0.9982888888888889
0.9996666666666667
0.9994666666666666
